# Main ideas

Let us denote as $P(\bar{w}, \bar{s})$ the probability to have $\bar{w}$ items waiting and $\bar{s}$ the items in service.

The equilibrium equations are as follows:

For $n>k$
$$\left (\sum_{i=1}^N \lambda_i +  \sum_{i=1}^N s_i\mu_i \right) P(\bar{w}, \bar{s}) = \sum_{i=1}^N \lambda_i P(\bar{w}-e_i, \bar{s}) + \sum_{i=1}^N \sum_{j=1}^N \frac{w_i+1}{|\bar{w}|+1} (s_j + 1 - e_{ij})\mu_j P(\bar{w}+e_i, \bar{s}-e_i+e_j)$$

For $n=k$
$$\left (\sum_{i=1}^N \lambda_i +  \sum_{i=1}^N s_i\mu_i \right) P(\bar{0}, \bar{s}) = \sum_{i=1}^N \lambda_i P(\bar{0}, \bar{s}-e_i) + \sum_{i=1}^N \sum_{j=1}^N (s_j + 1 - e_{ij})\mu_j P(e_i, \bar{s}-e_i+e_j)$$

For $0<n<k$
$$\left (\sum_{i=1}^N \lambda_i +  \sum_{i=1}^N s_i\mu_i \right) P(\bar{0}, \bar{s}) = \sum_{i=1}^N \lambda_i P(\bar{0}, \bar{s}-e_i) + \sum_{j=1}^N (s_j + 1)\mu_j P(0, \bar{s}+e_j)$$

For $0=n$
$$\sum_{i=1}^N \lambda_i P(\bar{0}, \bar{0}) = \sum_{j=1}^N \mu_j P(0, e_j)$$

The steady state probabilities P(\bar{w}, \bar{s}) can be expressed as:
$$P(\bar{w}, \bar{s}) =  |\bar{w}|!  \prod_{i=1}^N \frac{\alpha_i^{w_i}}{w_i!} P_n(\bar{s})$$

Denote further the vector that contains all  components $P_n(\bar{s})$ as $\mathbf{P}_n$ 


The equilibrium equations can be rewritten as:

For $n>k$
$$\mathbf{A}_0 \mathbf{P}_n = \mathbf{\Lambda} \mathbf{P}_{n-1} + \mathbf{A}_1 \mathbf{P}_{n+1}$$
where $\mathbf{A}_0$ is a diagonal matrix with $\sum_{i=1}^N \lambda_i +  \sum_{i=1}^N s_i\mu_i$ on each diagonal cell corresponding to $\bar{s}$. <br>In $\mathbf{A}_1$ each element corrsponding to $(\bar{s}, \bar{s}-e_i+e_j)$ will be:
$\sum_{i=1}^N \sum_{j=1}^N \alpha_i (s_j + 1 - e_{ij})\mu_j$ 

For $n=k$
$$\mathbf{A}_0 \mathbf{P}_{n} = \sum_{i=1}^N \lambda_i P(\bar{0}, \bar{s}-e_i) + \mathbf{A}_1 \mathbf{P}_{n+1}$$

For $0<n<k$
$$\left (\sum_{i=1}^N \lambda_i +  \sum_{i=1}^N s_i\mu_i \right) P(\bar{0}, \bar{s}) = \sum_{i=1}^N \lambda_i P(\bar{0}, \bar{s}-e_i) + \sum_{j=1}^N (s_j + 1)\mu_j P(0, \bar{s}+e_j)$$

For $0=n$
$$\sum_{i=1}^N \lambda_i P(\bar{0}, \bar{0}) = \sum_{j=1}^N \mu_j P(0, e_j)$$

In [2]:
# initialize libraries
import numpy as np
import itertools

In [3]:
# Define the generator that will generate all possible assignments of classes to servers, without permutations
def generateVectorsFixedSum(m,n):
    # generator for all combinations of $w$ for given number of servers and classes
    if m==1:
        yield [n]
    else:
        for i in range(n+1):
            for vect in generateVectorsFixedSum(m-1,n-i):
                yield [i]+vect

In [6]:
# Example
mClasses = 3
nServers = 2

# to produce the combinations
# the generators should be called in a loop
# the loop will stop when all possible combinations are generated
# the vectors contain numbers of customers of each class in the system
for vec in generateVectorsFixedSum(mClasses, nServers):
    print vec


[0, 0, 2]
[0, 1, 1]
[0, 2, 0]
[1, 0, 1]
[1, 1, 0]
[2, 0, 0]


In [38]:
mClasses = 1
lamda = np.linspace(1,2,mClasses)
mu = np.ones(mClasses) #np.linspace(2,1,mClasses)
nServers = 2
print lamda
print mu

[ 1.]
[ 1.]


In [39]:
lambdaTot = sum(lamda)
alpha = lamda/lambdaTot

idx_map = dict([ (tuple(vect), i) for i, vect in zip(itertools.count(), generateVectorsFixedSum(mClasses, nServers)) ])
i_map   = dict([(idx_map[idx], list(idx)) for idx in idx_map ])
q_max = len(idx_map)


In [37]:
def getIndexDict(idx, idx_map):
    try:
        return idx_map[tuple(idx)]
    except KeyError:
        return -1

In [40]:
A0 = np.zeros((q_max,q_max))  #corresponds to terms with i items in queue
A1 = np.zeros((q_max,q_max))  #corresponds to terms with i+1 items in queue
for i, idx in i_map.items():
    #diagonal term
    A0[i,i] += 1 + np.sum(idx*mu)/lambdaTot

    #term corresponding to end of service for item j1, start of service for j2
    for j1 in xrange(mClasses):
        for j2 in xrange(mClasses):
            idx[j1] += 1; idx[j2] -= 1
            i1 = getIndexDict(idx,idx_map)
            if i1>=0: A1[i,i1] += alpha[j2]/lambdaTot*idx[j1]*mu[j1]
            idx[j1] -= 1; idx[j2] += 1


In [41]:
eps = 0.000000001
I = np.eye(q_max)
Z_prev = np.zeros((q_max, q_max)) #I
delta=1
A0_inv = np.linalg.inv(A0)
while delta>eps:
    Z = np.dot(A0_inv, I + np.dot(A1, np.dot(Z_prev, Z_prev)))  #invA0*(I+A1*H*H)
    delta = np.sum(np.abs(Z-Z_prev))
    Z_prev=Z


In [42]:
Q = []
Q.insert(0, Z[:])

In [43]:
idx_map_nplus = dict([ (tuple(vect), i) for i, vect in zip(itertools.count(), generateVectorsFixedSum(mClasses, nServers)) ])
i_map_nplus   = dict([(idx_map_nplus[idx], list(idx)) for idx in idx_map_nplus ])
q_max_nplus   = len(idx_map_nplus)

idx_map_n = idx_map_nplus
i_map_n   = i_map_nplus
q_max_n   = q_max_nplus

A1_n = A1[:]

for n in range(nServers,0,-1):
    idx_map_nminus = dict([ (tuple(vect), i) for i, vect in zip(itertools.count(), generateVectorsFixedSum(mClasses, n-1)) ])
    i_map_nminus   = dict([(idx_map_nminus[idx], list(idx)) for idx in idx_map_nminus ])
    q_max_nminus   = len(idx_map_nminus)

    L_n = np.zeros((q_max_n,q_max_nminus))  #corresponds to terms with i items in queue
    A0_n = np.zeros((q_max_n,q_max_n))  #corresponds to terms with i items in queue
    for i, idx in i_map_n.items():

        #diagonal term
        A0_n[i,i] += 1 + np.sum(idx*mu)/lambdaTot

        #term corresponding to arrival of item item j1
        for j2 in xrange(mClasses):
            idx[j2] -= 1
            i2 = getIndexDict(idx,idx_map_nminus)
            if i2>=0: L_n[i,i2] += alpha[j2]
            idx[j2] += 1

    # Q_n = (A_0 - A_1*Q_{n+1})^{-1}*L_n
    Q.insert(0, np.dot(np.linalg.inv(A0_n-np.dot(A1_n, Q[0])), L_n))

    idx_map_nplus = idx_map_n
    i_map_nplus   = i_map_n
    q_max_nplus   = q_max_n

    idx_map_n = idx_map_nminus
    i_map_n   = i_map_nminus
    q_max_n   = q_max_nminus

    A1_n = np.zeros((q_max_n,q_max_nplus))  #corresponds to terms with i+1 items in queue
    for i, idx in i_map_n.items():
        #term corresponding to end of service for item j1
        for j1 in xrange(mClasses):
            idx[j1] += 1
            i1 = getIndexDict(idx,idx_map_nplus)
            if i1>=0: A1_n[i,i1] += idx[j1]*mu[j1]/lambdaTot
            idx[j1] -= 1
print Q

[array([[ 1.]]), array([[ 0.5]]), array([[ 0.5]])]


In [45]:
P = []
P.append([1.0])

sm = 1.0
for n in xrange(nServers):
    P.append(np.dot(Q[n],P[-1]))
    sm += sum(P[-1])

sm += sum(np.dot(np.linalg.inv(np.eye(len(P[-1])) - Z), P[-1]))

for p in P: p /= sm  #normalization
print sm, P

3.49999998849 [[1.0], array([ 0.28571429]), array([ 0.14285714])]


In [31]:
inv1minZ = np.linalg.inv(np.eye(len(P[-1])) - Z)
EQTotal = sum(np.dot(np.dot(np.dot(inv1minZ,inv1minZ), Z),P[-1]))
EQQmin1Total = 2*sum(np.dot(np.dot(np.dot(np.dot(np.dot(inv1minZ,inv1minZ),inv1minZ), Z), Z), P[-1]))
EQ2Total = EQQmin1Total + EQTotal

EQ = alpha*EQTotal
EQQmin1 = alpha*alpha*EQQmin1Total
EQ2 = EQQmin1 + EQ

In [32]:
VarQTotal = EQ2Total - EQTotal**2
VarQ = EQ2 - EQ**2

In [108]:
np.dot(A1, np.dot(Z_prev, Z_prev))

array([[ 4.]])

In [109]:
np.dot(A0_inv, I + np.dot(A1, np.dot(Z_prev, Z_prev)))

array([[ 1.]])

In [86]:
I

array([[1, 0],
       [0, 1]])

In [87]:
q_max

1

In [126]:
Z

array([[ 0.88457248,  0.21720737,  0.1091813 ],
       [ 0.15875699,  0.66024058,  0.17796341],
       [ 0.0172222 ,  0.02988751,  0.4853448 ]])